In [5]:
#Before runnig the program you need to change the location line

#Function to normalize:
    
def NormalizedNumbersToField(table, field, scratchGDB):
    import arcpy
    import os
    field_NORMALIZED = field + "_NORMALIZED"
    arcpy.AddField_management(table,field_NORMALIZED , "DOUBLE")
    scratchTable = os.path.join(scratchGDB, "Temp_Feat")
    arcpy.Statistics_analysis(table,scratchTable, [[str(field),"MIN"],[str(field),"MAX"]])
    with arcpy.da.SearchCursor(scratchTable, ("MAX_" + str(field) , "MIN_" + str(field))) as cursor:
        for row in cursor:
            maxNum = row[0]
            minNum = row[1]
    del cursor, row
    arcpy.Delete_management(scratchTable)
    with arcpy.da.UpdateCursor(table, ( field , field_NORMALIZED )) as cursor:
        for row in cursor:
            number = row[0]
            row[1] = (( number - minNum ) / ( maxNum - minNum ))
            cursor.updateRow(row)
    del cursor, row

def covid():
    
    #Import arcpy, os and create workspace:
    import arcpy
    import os
    location = "C:\\Users\\Angel Naranjo\\Desktop\\COVID\\COVID.gdb"
    arcpy.env.workspace = location
    arcpy.Exists(arcpy.env.workspace)
    arcpy.env.overwriteOutput = True
    
    # Count the number of hospitals in each province:
    # Create provinces3 with the data of Provinces2 and the number of hospitals in each one
    target_features = "Provinces2"
    join_features = "Hospitals"
    out_feature_class = "Provinces3"
    join_operation = "JOIN_ONE_TO_ONE"
    join_type =  "KEEP_ALL"
    field_mapping = ""
    match_option = "INTERSECT"
    search_radius = ""
    distance_field_name = ""
    arcpy.SpatialJoin_analysis (target_features, join_features, out_feature_class,join_operation,join_type,
                      field_mapping, match_option, search_radius , distance_field_name)
    
    #Rename Join_Count as Hospitals:
    fc = location + "\\Provinces3"
    field = "Join_Count" 
    new_name = "Hospitals"
    new_alias = "Hospitals"
    new_type = ""
    new_length = "60"
    new_is_nullable = "NULLABLE"
    clear_alias = "FALSE"
    arcpy.AlterField_management(fc, field, new_name, new_alias, new_type, new_length, new_is_nullable,clear_alias)
    
    # Count the number of nursing homes in each province:
    # Create provinces4 with the data of Provinces3 and the number of nursing homes in each one
    target_features = "Provinces3"
    join_features = "NursingHomes"
    out_feature_class = "Provinces4"
    join_operation = "JOIN_ONE_TO_ONE"
    join_type =  "KEEP_ALL"
    field_mapping = ""
    match_option = "INTERSECT"
    search_radius = ""
    distance_field_name = ""
    arcpy.SpatialJoin_analysis (target_features, join_features, out_feature_class,join_operation,join_type,
                      field_mapping, match_option, search_radius , distance_field_name)
    
    #Rename Point_Count as NursingHomes:
    fc = location + "\\Provinces4"
    field = "Join_Count" 
    new_name = "NursingHomes"
    new_alias = "NursingHomes"
    new_type = ""
    new_length = "60"
    new_is_nullable = "NULLABLE"
    clear_alias = "FALSE"
    arcpy.AlterField_management(fc, field, new_name, new_alias, new_type, new_length, new_is_nullable, clear_alias)
    
    # Add field: Total population:
    inFeatures = "Provinces4"
    field_name = "Total_population"
    field_type = "FLOAT"
    arcpy.AddField_management(inFeatures, field_name, field_type)
    
    # Add field: Transmission risk:
    inFeatures = "Provinces4"
    field_name = "Transsmission_risk"
    field_type = "FLOAT"
    arcpy.AddField_management(inFeatures, field_name, field_type)
    
    # Add field: Death Rate:
    inFeatures = "Provinces4"
    field_name = "Death_rate"
    field_type = "FLOAT"
    arcpy.AddField_management(inFeatures, field_name, field_type)
    
    # Add field: Total risk:
    inFeatures = "Provinces4"
    field_name = "Total_risk"
    field_type = "FLOAT"
    arcpy.AddField_management(inFeatures, field_name, field_type)
    
    #Calculate Total population:
    in_table = "Provinces4"
    field = "Total_population"
    expression = "!Total14! + !Total25! + !Total49! + !Total65! + !Total100!"
    expression_type = "PYTHON3"
    code_block = ""
    field_type = "TEXT"
    arcpy.CalculateField_management(in_table, field, expression, expression_type, code_block, 
                                    field_type)
    
    #Calculate Transmission risk:
    in_table = "Provinces4"
    field = "Transsmission_risk"
    expression = "!Density! * !Total49! / !Total_population!"
    expression_type = "PYTHON3"
    code_block = ""
    field_type = "TEXT"
    arcpy.CalculateField_management(in_table, field, expression, expression_type, code_block, 
                                    field_type)
    
    #Calculate Death rate:
    in_table = "Provinces4"
    field = "Death_rate"
    expression = "!NursingHomes! * !Transsmission_risk! * !Total100! / !Hospitals!"
    expression_type = "PYTHON3"
    code_block = ""
    field_type = "TEXT"
    arcpy.CalculateField_management(in_table, field, expression, expression_type, code_block,
                                    field_type)
    
    #Calculate Total risk:
    in_table = "Provinces4"
    field = "Total_risk"
    expression = "!Death_rate! * !Transsmission_risk! * !Total100! / !Total14!"
    expression_type = "PYTHON3"
    code_block = ""
    field_type = "TEXT"
    arcpy.CalculateField_management(in_table, field, expression, expression_type, code_block,
                                    field_type)
    
        
    #Normalize Transmission risk:
    NormalizedNumbersToField("Provinces4", "Transsmission_risk",location)
    
    #Normalize Death rate:
    NormalizedNumbersToField("Provinces4", "Death_rate",location)
    
    #Normalize Total risk:
    NormalizedNumbersToField("Provinces4", "Total_risk",location)

In [6]:
covid()